In [8]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, monotonically_increasing_id

# Khởi tạo phiên Spark với MongoDB và PostgreSQL

spark = SparkSession.builder \
    .appName("Goodreads Spark with MongoDB and PostgreSQL") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1,org.postgresql:postgresql:42.7.4") \
    .config("spark.mongodb.read.connection.uri", "mongodb://localhost:27017/db_BDS.coll_BDS") \
    .getOrCreate()

24/10/10 09:54:28 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [9]:
# Cài đặt mức độ log

spark.sparkContext.setLogLevel("INFO")

# Đọc dữ liệu từ MongoDB vào DataFrame

df = spark.read \
    .format("mongo") \
    .option("uri", "mongodb://localhost:27017/db_BDS.coll_BDS") \
    .load()

df.show(5)
df.printSchema()

24/10/10 09:54:31 INFO MemoryStore: Block broadcast_10 stored as values in memory (estimated size 200.0 B, free 434.3 MiB)
24/10/10 09:54:31 INFO MemoryStore: Block broadcast_10_piece0 stored as bytes in memory (estimated size 408.0 B, free 434.3 MiB)
24/10/10 09:54:31 INFO BlockManagerInfo: Added broadcast_10_piece0 in memory on 192.168.34.128:59436 (size: 408.0 B, free: 434.4 MiB)
24/10/10 09:54:31 INFO SparkContext: Created broadcast 10 from broadcast at MongoSpark.scala:530
24/10/10 09:54:31 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 09:54:31 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 09:54:31 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 09:54:31 INFO connection: Opened connection [connectionId{localValue:9, serverValue:301}] to localhost:27017
24/10/10 09:54:31 INFO cluster: Monitor thr

+---------+--------------+------+--------------------+----------+--------------+----------+------------+---------------+-----------+--------------------+--------------------+--------------------+-----------------+
|Diện_tích|           Giá|Mã_tin|               Mô_tả| Ngày_đăng|       Pháp_lý|      Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|                 _id|       chi_tiet_link|            Đường|
+---------+--------------+------+--------------------+----------+--------------+----------+------------+---------------+-----------+--------------------+--------------------+--------------------+-----------------+
|    80 m²|4 tỷ 700 triệu|121248|      Thông tin nhà:|06/10/2024|          NULL|   Tân Phú|     2 phòng|        2 phòng|Hồ Chí Minh|Bán nhà chính chủ...|{670721903073fcca...|https://batdongsa...|        Phú Trung|
|    70 m²|5 tỷ 800 triệu|121204|Nhà  Nguyễn Văn Đ...|02/10/2024|Sổ đỏ/ Sổ hồng|Bình Thạnh|     2 phòng|        2 phòng|Hồ Chí Minh|Nhà  Nguyễn 

24/10/10 09:54:31 INFO connection: Opened connection [connectionId{localValue:14, serverValue:306}] to localhost:27017
24/10/10 09:54:31 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireVersion=17, maxDocumentSize=16777216, logicalSessionTimeoutMinutes=30, roundTripTimeNanos=712209}
24/10/10 09:54:31 INFO connection: Opened connection [connectionId{localValue:15, serverValue:307}] to localhost:27017
24/10/10 09:54:31 INFO MongoClientCache: Closing MongoClient: [localhost:27017]
24/10/10 09:54:31 INFO connection: Closed connection [connectionId{localValue:15, serverValue:307}] to localhost:27017 because the pool has been closed.
24/10/10 09:54:31 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 09:54:31 INFO MongoClientCache: Creating MongoCli

In [10]:
#Xóa các cột không cần thiết
df = df.drop("_id","Chi_tiet_link")
# Hiển thị dữ liệu sau khi xóa cột
df.show()

+---------+---------------+------+--------------------+----------+--------------+-----------+------------+---------------+-----------+--------------------+-------------------+
|Diện_tích|            Giá|Mã_tin|               Mô_tả| Ngày_đăng|       Pháp_lý|       Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|              Đường|
+---------+---------------+------+--------------------+----------+--------------+-----------+------------+---------------+-----------+--------------------+-------------------+
|    80 m²| 4 tỷ 700 triệu|121248|      Thông tin nhà:|06/10/2024|          NULL|    Tân Phú|     2 phòng|        2 phòng|Hồ Chí Minh|Bán nhà chính chủ...|          Phú Trung|
|    70 m²| 5 tỷ 800 triệu|121204|Nhà  Nguyễn Văn Đ...|02/10/2024|Sổ đỏ/ Sổ hồng| Bình Thạnh|     2 phòng|        2 phòng|Hồ Chí Minh|Nhà  Nguyễn Văn Đ...|     Nguyễn Văn Đậu|
|    45 m²| 9 tỷ 800 triệu|121156|BÁN NHÀ NGUYỄN KH...|02/10/2024|Sổ đỏ/ Sổ hồng|   Cầu Giấy|     5 phòng|        3 phòn

24/10/10 09:55:28 INFO MongoRelation: requiredColumns: Diện_tích, Giá, Mã_tin, Mô_tả, Ngày_đăng, Pháp_lý, Quận, Số_phòng_ngủ, Số_phòng_toilet, Thành_phố, Tiêu_đề, Đường, filters: 
24/10/10 09:55:28 INFO CodeGenerator: Code generated in 8.272333 ms
24/10/10 09:55:28 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 09:55:28 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 09:55:28 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 09:55:28 INFO connection: Opened connection [connectionId{localValue:21, serverValue:313}] to localhost:27017
24/10/10 09:55:28 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireVersion=17, maxDocumentSize=16777216, logicalSessionTimeoutMinutes=30

24/10/10 09:55:28 INFO BlockManagerInfo: Removed broadcast_9_piece0 on 192.168.34.128:59436 in memory (size: 15.0 KiB, free: 434.4 MiB)


In [11]:
df.columns

['Diện_tích',
 'Giá',
 'Mã_tin',
 'Mô_tả',
 'Ngày_đăng',
 'Pháp_lý',
 'Quận',
 'Số_phòng_ngủ',
 'Số_phòng_toilet',
 'Thành_phố',
 'Tiêu_đề',
 'Đường']

In [12]:
from pyspark.sql import functions as F

def remove_special_characters_pyspark(df, text_column):
    # Xử lý biểu tượng emoji và ký tự đặc biệt
    emoji_pattern = "[" \
        "\U0001F600-\U0001F64F"  \
        "\U0001F300-\U0001F5FF"  \
        "\U0001F680-\U0001F6FF"  \
        "\U0001F1E0-\U0001F1FF"  \
        "\u2600-\u26FF"  \
        "\u2700-\u27BF"  \
        "]"
    
    # Loại bỏ emoji
    df = df.withColumn(text_column, F.regexp_replace(text_column, emoji_pattern, ""))
    
    # Loại bỏ các ký tự đặc biệt, chỉ giữ lại các ký tự tiếng Việt, chữ cái, số, khoảng trắng và dấu câu cơ bản
    allowed_chars = r'[^a-zA-Z0-9\s.,áàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệíìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữựýỳỷỹỵđÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬÉÈẺẼẸÊẾỀỂỄỆÍÌỈĨỊÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÚÙỦŨỤƯỨỪỬỮỰÝỲỶỸỴĐ]'
    
    df = df.withColumn(text_column, F.regexp_replace(text_column, allowed_chars, ""))
    
    return df
# Giả sử DataFrame có cột 'text' chứa dữ liệu văn bản
df=remove_special_characters_pyspark(df, 'Mô_tả')
df=remove_special_characters_pyspark(df, 'Tiêu_đề')


In [13]:
# kiểm tra lại
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

# Biểu thức regex cho emoji
emoji_pattern = "[" \
    "\U0001F600-\U0001F64F"  \
    "\U0001F300-\U0001F5FF"  \
    "\U0001F680-\U0001F6FF"  \
    "\U0001F1E0-\U0001F1FF"  \
    "\u2600-\u26FF"  \
    "\u2700-\u27BF"  \
    "]"

# Hàm Python kiểm tra xem chuỗi có chứa emoji không
def contains_emoji(text):
    if re.search(emoji_pattern, text):
        return True
    return False

# Đăng ký hàm UDF trong PySpark
contains_emoji_udf = udf(contains_emoji, BooleanType())


In [14]:
# Áp dụng UDF để kiểm tra emoji trong cột 'Mô_tả' và 'Tiêu_đề'
df_with_emoji = df.withColumn('has_emoji_in_Mo_ta', contains_emoji_udf(df['Mô_tả'])) \
                  .withColumn('has_emoji_in_Tieu_de', contains_emoji_udf(df['Tiêu_đề']))

# Lọc những hàng chứa emoji
emoji_rows = df_with_emoji.filter(F.col('has_emoji_in_Mo_ta') | F.col('has_emoji_in_Tieu_de'))

# Đếm số hàng chứa emoji
emoji_count = emoji_rows.count()
print(f"Số hàng chứa emoji: {emoji_count}")


24/10/10 09:56:21 INFO MongoRelation: requiredColumns: Mô_tả, Tiêu_đề, filters: 
24/10/10 09:56:21 INFO CodeGenerator: Code generated in 7.309542 ms
24/10/10 09:56:21 INFO CodeGenerator: Code generated in 4.918958 ms
24/10/10 09:56:21 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 09:56:21 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 09:56:21 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 09:56:21 INFO connection: Opened connection [connectionId{localValue:28, serverValue:320}] to localhost:27017
24/10/10 09:56:21 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireVersion=17, maxDocumentSize=16777216, logicalSessionTimeoutMinutes=30, roundTripTimeNanos=205625}
24

Số hàng chứa emoji: 0


24/10/10 09:56:22 INFO Executor: Finished task 0.0 in stage 10.0 (TID 10). 2669 bytes result sent to driver
24/10/10 09:56:22 INFO TaskSetManager: Finished task 0.0 in stage 10.0 (TID 10) in 701 ms on 192.168.34.128 (executor driver) (1/1)
24/10/10 09:56:22 INFO TaskSchedulerImpl: Removed TaskSet 10.0, whose tasks have all completed, from pool 
24/10/10 09:56:22 INFO PythonAccumulatorV2: Connected to AccumulatorServer at host: 127.0.0.1 port: 59437
24/10/10 09:56:22 INFO DAGScheduler: ShuffleMapStage 10 (count at NativeMethodAccessorImpl.java:0) finished in 0,713 s
24/10/10 09:56:22 INFO DAGScheduler: looking for newly runnable stages
24/10/10 09:56:22 INFO DAGScheduler: running: Set()
24/10/10 09:56:22 INFO DAGScheduler: waiting: Set()
24/10/10 09:56:22 INFO DAGScheduler: failed: Set()
24/10/10 09:56:22 INFO CodeGenerator: Code generated in 3.166459 ms
24/10/10 09:56:22 INFO SparkContext: Starting job: count at NativeMethodAccessorImpl.java:0
24/10/10 09:56:22 INFO DAGScheduler: Got j

In [15]:
df.show()

+---------+---------------+------+--------------------+----------+--------------+-----------+------------+---------------+-----------+--------------------+-------------------+
|Diện_tích|            Giá|Mã_tin|               Mô_tả| Ngày_đăng|       Pháp_lý|       Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|              Đường|
+---------+---------------+------+--------------------+----------+--------------+-----------+------------+---------------+-----------+--------------------+-------------------+
|    80 m²| 4 tỷ 700 triệu|121248|       Thông tin nhà|06/10/2024|          NULL|    Tân Phú|     2 phòng|        2 phòng|Hồ Chí Minh|Bán nhà chính chủ...|          Phú Trung|
|    70 m²| 5 tỷ 800 triệu|121204|Nhà  Nguyễn Văn Đ...|02/10/2024|Sổ đỏ/ Sổ hồng| Bình Thạnh|     2 phòng|        2 phòng|Hồ Chí Minh|Nhà  Nguyễn Văn Đ...|     Nguyễn Văn Đậu|
|    45 m²| 9 tỷ 800 triệu|121156|BÁN NHÀ NGUYỄN KH...|02/10/2024|Sổ đỏ/ Sổ hồng|   Cầu Giấy|     5 phòng|        3 phòn

24/10/10 09:56:33 INFO MongoRelation: requiredColumns: Diện_tích, Giá, Mã_tin, Mô_tả, Ngày_đăng, Pháp_lý, Quận, Số_phòng_ngủ, Số_phòng_toilet, Thành_phố, Tiêu_đề, Đường, filters: 
24/10/10 09:56:33 INFO CodeGenerator: Code generated in 8.864792 ms
24/10/10 09:56:33 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 09:56:33 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 09:56:33 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 09:56:33 INFO connection: Opened connection [connectionId{localValue:35, serverValue:327}] to localhost:27017
24/10/10 09:56:33 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireVersion=17, maxDocumentSize=16777216, logicalSessionTimeoutMinutes=30

24/10/10 09:56:34 INFO BlockManagerInfo: Removed broadcast_15_piece0 on 192.168.34.128:59436 in memory (size: 5.9 KiB, free: 434.4 MiB)
24/10/10 09:56:34 INFO BlockManagerInfo: Removed broadcast_14_piece0 on 192.168.34.128:59436 in memory (size: 18.6 KiB, free: 434.4 MiB)


In [16]:
from pyspark.sql import functions as F

def remove_text_pyspark(df, text_column):
    # Loại bỏ tất cả ký tự không phải là chữ số
    df = df.withColumn(text_column, F.regexp_replace(text_column, r'[^\d]', ''))
    return df
df=remove_text_pyspark(df,'Diện_tích')
df=remove_text_pyspark(df,'Số_phòng_ngủ')
df=remove_text_pyspark(df,'Số_phòng_toilet')


In [17]:
from pyspark.sql import functions as F

# Lọc các hàng có giá trị không phải là số trong cột 'Diện_tích'
non_numeric_dien_tich = df.filter(F.col('Diện_tích').rlike(r'[^\d]'))

# Lọc các hàng có giá trị không phải là số trong cột 'Số_phòng_ngủ'
non_numeric_so_phong_ngu = df.filter(F.col('Số_phòng_ngủ').rlike(r'[^\d]'))

# Lọc các hàng có giá trị không phải là số trong cột 'Số_phòng_toilet'
non_numeric_so_phong_toilet = df.filter(F.col('Số_phòng_toilet').rlike(r'[^\d]'))

# Đếm số lượng hàng chứa ký tự không phải số trong từng cột
non_numeric_dien_tich_count = non_numeric_dien_tich.count()
non_numeric_so_phong_ngu_count = non_numeric_so_phong_ngu.count()
non_numeric_so_phong_toilet_count = non_numeric_so_phong_toilet.count()

print(f"Số hàng có giá trị không phải số trong cột 'Diện_tích': {non_numeric_dien_tich_count}")
print(f"Số hàng có giá trị không phải số trong cột 'Số_phòng_ngủ': {non_numeric_so_phong_ngu_count}")
print(f"Số hàng có giá trị không phải số trong cột 'Số_phòng_toilet': {non_numeric_so_phong_toilet_count}")


24/10/10 09:57:03 INFO MongoRelation: requiredColumns: Diện_tích, filters: IsNotNull(Diện_tích)
24/10/10 09:57:03 INFO CodeGenerator: Code generated in 8.841333 ms
24/10/10 09:57:03 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 09:57:03 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 09:57:03 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 09:57:03 INFO connection: Opened connection [connectionId{localValue:42, serverValue:334}] to localhost:27017
24/10/10 09:57:03 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireVersion=17, maxDocumentSize=16777216, logicalSessionTimeoutMinutes=30, roundTripTimeNanos=295917}
24/10/10 09:57:03 INFO connection: Opened connection [c

Số hàng có giá trị không phải số trong cột 'Diện_tích': 0
Số hàng có giá trị không phải số trong cột 'Số_phòng_ngủ': 0
Số hàng có giá trị không phải số trong cột 'Số_phòng_toilet': 0


24/10/10 09:57:04 INFO DAGScheduler: Got job 16 (count at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/10/10 09:57:04 INFO DAGScheduler: Final stage: ResultStage 19 (count at NativeMethodAccessorImpl.java:0)
24/10/10 09:57:04 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 18)
24/10/10 09:57:04 INFO DAGScheduler: Missing parents: List()
24/10/10 09:57:04 INFO DAGScheduler: Submitting ResultStage 19 (MapPartitionsRDD[81] at count at NativeMethodAccessorImpl.java:0), which has no missing parents
24/10/10 09:57:04 INFO MemoryStore: Block broadcast_20 stored as values in memory (estimated size 12.5 KiB, free 434.3 MiB)
24/10/10 09:57:04 INFO MemoryStore: Block broadcast_20_piece0 stored as bytes in memory (estimated size 5.9 KiB, free 434.3 MiB)
24/10/10 09:57:04 INFO BlockManagerInfo: Added broadcast_20_piece0 in memory on 192.168.34.128:59436 (size: 5.9 KiB, free: 434.4 MiB)
24/10/10 09:57:04 INFO SparkContext: Created broadcast 20 from broadcast at DAGSch

In [18]:
df.printSchema()

root
 |-- Diện_tích: string (nullable = true)
 |-- Giá: string (nullable = true)
 |-- Mã_tin: string (nullable = true)
 |-- Mô_tả: string (nullable = true)
 |-- Ngày_đăng: string (nullable = true)
 |-- Pháp_lý: string (nullable = true)
 |-- Quận: string (nullable = true)
 |-- Số_phòng_ngủ: string (nullable = true)
 |-- Số_phòng_toilet: string (nullable = true)
 |-- Thành_phố: string (nullable = true)
 |-- Tiêu_đề: string (nullable = true)
 |-- Đường: string (nullable = true)



In [19]:
from pyspark.sql import functions as F
from pyspark.sql.types import LongType

# Định nghĩa hàm xử lý giá
def xu_ly_gia(price_str):
    if price_str is None:  # Kiểm tra giá trị None
        return 0  # Hoặc bạn có thể trả về một giá trị khác nếu cần

    units = {
        'tỷ': 1000000000,
        'triệu': 1000000,
        'nghìn': 1000,
    }

    total = 0
    current_value = 0
    parts = price_str.split()
    
    for part in parts:
        if part in units:
            total += current_value * units[part]
            current_value = 0
        else:
            try:
                current_value = int(part)
            except ValueError:
                current_value = 0  # Trường hợp không thể chuyển đổi
    total += current_value
    return total

# Chuyển hàm Python thành UDF trong PySpark
xu_ly_gia_udf = F.udf(xu_ly_gia, LongType())  # Sử dụng LongType để tương thích

# Áp dụng UDF lên cột 'Giá' trong DataFrame và thay đổi cột 'Giá'
df = df.withColumn('Giá', xu_ly_gia_udf(F.col('Giá')))

# Hiển thị DataFrame sau khi chuyển đổi
df.show()


+---------+-----------+------+--------------------+----------+--------------+-----------+------------+---------------+-----------+--------------------+-------------------+
|Diện_tích|        Giá|Mã_tin|               Mô_tả| Ngày_đăng|       Pháp_lý|       Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|              Đường|
+---------+-----------+------+--------------------+----------+--------------+-----------+------------+---------------+-----------+--------------------+-------------------+
|       80| 4700000000|121248|       Thông tin nhà|06/10/2024|          NULL|    Tân Phú|           2|              2|Hồ Chí Minh|Bán nhà chính chủ...|          Phú Trung|
|       70| 5800000000|121204|Nhà  Nguyễn Văn Đ...|02/10/2024|Sổ đỏ/ Sổ hồng| Bình Thạnh|           2|              2|Hồ Chí Minh|Nhà  Nguyễn Văn Đ...|     Nguyễn Văn Đậu|
|       45| 9800000000|121156|BÁN NHÀ NGUYỄN KH...|02/10/2024|Sổ đỏ/ Sổ hồng|   Cầu Giấy|           5|              3|     Hà Nội|BÁN NHÀ NG

24/10/10 09:57:55 INFO MongoRelation: requiredColumns: Diện_tích, Giá, Mã_tin, Mô_tả, Ngày_đăng, Pháp_lý, Quận, Số_phòng_ngủ, Số_phòng_toilet, Thành_phố, Tiêu_đề, Đường, filters: 
24/10/10 09:57:55 INFO CodeGenerator: Code generated in 11.897625 ms
24/10/10 09:57:55 INFO CodeGenerator: Code generated in 3.60275 ms
24/10/10 09:57:55 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 09:57:55 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 09:57:55 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 09:57:55 INFO connection: Opened connection [connectionId{localValue:63, serverValue:355}] to localhost:27017
24/10/10 09:57:55 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireV

In [20]:
from pyspark.sql.functions import col, to_date
from pyspark.sql.types import IntegerType, FloatType, LongType

# Chuyển đổi kiểu dữ liệu
df = df.withColumn("Diện_tích", col("Diện_tích").cast(FloatType())) \
       .withColumn("Giá", col("Giá").cast(LongType()))\
       .withColumn("Số_phòng_ngủ", col("Số_phòng_ngủ").cast(IntegerType())) \
       .withColumn("Số_phòng_toilet", col("Số_phòng_toilet").cast(IntegerType())) \
       .withColumn("Ngày_đăng", to_date(col("Ngày_đăng"), "dd/MM/yyyy"))

# Hiển thị schema sau khi xử lý
df.printSchema()

# Hiển thị dữ liệu sau khi xử lý
df.show()


root
 |-- Diện_tích: float (nullable = true)
 |-- Giá: long (nullable = true)
 |-- Mã_tin: string (nullable = true)
 |-- Mô_tả: string (nullable = true)
 |-- Ngày_đăng: date (nullable = true)
 |-- Pháp_lý: string (nullable = true)
 |-- Quận: string (nullable = true)
 |-- Số_phòng_ngủ: integer (nullable = true)
 |-- Số_phòng_toilet: integer (nullable = true)
 |-- Thành_phố: string (nullable = true)
 |-- Tiêu_đề: string (nullable = true)
 |-- Đường: string (nullable = true)

+---------+-----------+------+--------------------+----------+--------------+-----------+------------+---------------+-----------+--------------------+-------------------+
|Diện_tích|        Giá|Mã_tin|               Mô_tả| Ngày_đăng|       Pháp_lý|       Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|              Đường|
+---------+-----------+------+--------------------+----------+--------------+-----------+------------+---------------+-----------+--------------------+-------------------+
|     

24/10/10 09:58:14 INFO MongoRelation: requiredColumns: Diện_tích, Giá, Mã_tin, Mô_tả, Ngày_đăng, Pháp_lý, Quận, Số_phòng_ngủ, Số_phòng_toilet, Thành_phố, Tiêu_đề, Đường, filters: 
24/10/10 09:58:14 INFO BlockManagerInfo: Removed broadcast_23_piece0 on 192.168.34.128:59436 in memory (size: 22.2 KiB, free: 434.4 MiB)
24/10/10 09:58:14 INFO BlockManagerInfo: Removed broadcast_22_piece0 on 192.168.34.128:59436 in memory (size: 5.9 KiB, free: 434.4 MiB)
24/10/10 09:58:14 INFO CodeGenerator: Code generated in 23.418417 ms
24/10/10 09:58:14 INFO CodeGenerator: Code generated in 3.538041 ms
24/10/10 09:58:14 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 09:58:14 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 09:58:14 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 09:58:14 INFO connection: Opened connection [

In [21]:
# Lấy các giá trị khác nhau trong cột 'Pháp_lý'
distinct_phap_ly_values = df.select('Pháp_lý').distinct()

# Hiển thị các giá trị khác nhau
distinct_phap_ly_values.show(truncate=False)


24/10/10 09:58:35 INFO MongoRelation: requiredColumns: Pháp_lý, filters: 
24/10/10 09:58:35 INFO CodeGenerator: Code generated in 36.627375 ms
24/10/10 09:58:35 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 09:58:35 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 09:58:35 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 09:58:35 INFO connection: Opened connection [connectionId{localValue:77, serverValue:369}] to localhost:27017
24/10/10 09:58:35 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireVersion=17, maxDocumentSize=16777216, logicalSessionTimeoutMinutes=30, roundTripTimeNanos=215000}
24/10/10 09:58:35 INFO connection: Opened connection [connectionId{localValu

+----------------+
|Pháp_lý         |
+----------------+
|Hợp đồng mua bán|
|Sổ đỏ/ Sổ hồng  |
|NULL            |
+----------------+



24/10/10 09:58:35 INFO CodeGenerator: Code generated in 8.659083 ms
24/10/10 09:58:35 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/10/10 09:58:35 INFO DAGScheduler: Got job 22 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/10/10 09:58:35 INFO DAGScheduler: Final stage: ResultStage 27 (showString at NativeMethodAccessorImpl.java:0)
24/10/10 09:58:35 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 26)
24/10/10 09:58:35 INFO DAGScheduler: Missing parents: List()
24/10/10 09:58:35 INFO DAGScheduler: Submitting ResultStage 27 (MapPartitionsRDD[113] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/10/10 09:58:35 INFO MemoryStore: Block broadcast_26 stored as values in memory (estimated size 46.8 KiB, free 434.2 MiB)
24/10/10 09:58:35 INFO MemoryStore: Block broadcast_26_piece0 stored as bytes in memory (estimated size 21.2 KiB, free 434.2 MiB)
24/10/10 09:58:35 INFO BlockManage

In [22]:
from pyspark.sql import functions as F

# Tích hợp phần trước ký tự '/' trong cột 'Pháp_lý'
df = df.withColumn('Pháp_lý', F.split(F.col('Pháp_lý'), '/').getItem(0))


24/10/10 09:58:46 INFO BlockManagerInfo: Removed broadcast_25_piece0 on 192.168.34.128:59436 in memory (size: 18.3 KiB, free: 434.4 MiB)
24/10/10 09:58:46 INFO BlockManagerInfo: Removed broadcast_26_piece0 on 192.168.34.128:59436 in memory (size: 21.2 KiB, free: 434.4 MiB)
24/10/10 09:58:46 INFO BlockManagerInfo: Removed broadcast_24_piece0 on 192.168.34.128:59436 in memory (size: 24.0 KiB, free: 434.4 MiB)


In [23]:
# kiểm tra lại 
# Lấy các giá trị khác nhau trong cột 'Pháp_lý'
distinct_phap_ly_values = df.select('Pháp_lý').distinct()

# Hiển thị các giá trị khác nhau
distinct_phap_ly_values.show(truncate=False)


+----------------+
|Pháp_lý         |
+----------------+
|Hợp đồng mua bán|
|Sổ đỏ           |
|NULL            |
+----------------+



24/10/10 09:58:59 INFO MongoRelation: requiredColumns: Pháp_lý, filters: 
24/10/10 09:58:59 INFO CodeGenerator: Code generated in 9.346291 ms
24/10/10 09:58:59 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 09:58:59 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 09:58:59 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 09:58:59 INFO connection: Opened connection [connectionId{localValue:84, serverValue:376}] to localhost:27017
24/10/10 09:58:59 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireVersion=17, maxDocumentSize=16777216, logicalSessionTimeoutMinutes=30, roundTripTimeNanos=408792}
24/10/10 09:58:59 INFO connection: Opened connection [connectionId{localValue

In [24]:
# Tạo danh sách các cột kiểm tra giá trị null để dễ dàng xử lý
columns_to_check = ['Diện_tích', 'Giá', 'Mã_tin', 'Mô_tả', 'Ngày_đăng', 'Pháp_lý', 'Quận', 'Số_phòng_ngủ', 'Số_phòng_toilet', 'Thành_phố', 'Tiêu_đề', 'Đường']

# Lặp qua từng cột và đếm số giá trị null
for col in columns_to_check:
    null_count = df.filter(F.col(col).isNull()).count()  # Đếm số giá trị null trong cột
    print(f"Số giá trị null trong cột '{col}': {null_count}")


24/10/10 09:59:23 INFO MongoRelation: requiredColumns: Diện_tích, filters: 
24/10/10 09:59:23 INFO CodeGenerator: Code generated in 8.959709 ms
24/10/10 09:59:23 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 09:59:23 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 09:59:23 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 09:59:23 INFO connection: Opened connection [connectionId{localValue:91, serverValue:383}] to localhost:27017
24/10/10 09:59:23 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireVersion=17, maxDocumentSize=16777216, logicalSessionTimeoutMinutes=30, roundTripTimeNanos=182625}
24/10/10 09:59:23 INFO connection: Opened connection [connectionId{localVal

Số giá trị null trong cột 'Diện_tích': 33
Số giá trị null trong cột 'Giá': 0


24/10/10 09:59:24 INFO PythonUDFRunner: Times: total = 58, boot = 9, init = 47, finish = 2
24/10/10 09:59:24 INFO MongoClientCache: Closing MongoClient: [localhost:27017]
24/10/10 09:59:24 INFO connection: Closed connection [connectionId{localValue:104, serverValue:396}] to localhost:27017 because the pool has been closed.
24/10/10 09:59:24 INFO Executor: Finished task 0.0 in stage 34.0 (TID 27). 2712 bytes result sent to driver
24/10/10 09:59:24 INFO TaskSetManager: Finished task 0.0 in stage 34.0 (TID 27) in 99 ms on 192.168.34.128 (executor driver) (1/1)
24/10/10 09:59:24 INFO TaskSchedulerImpl: Removed TaskSet 34.0, whose tasks have all completed, from pool 
24/10/10 09:59:24 INFO DAGScheduler: ShuffleMapStage 34 (count at NativeMethodAccessorImpl.java:0) finished in 0,108 s
24/10/10 09:59:24 INFO DAGScheduler: looking for newly runnable stages
24/10/10 09:59:24 INFO DAGScheduler: running: Set()
24/10/10 09:59:24 INFO DAGScheduler: waiting: Set()
24/10/10 09:59:24 INFO DAGScheduler

Số giá trị null trong cột 'Mã_tin': 0
Số giá trị null trong cột 'Mô_tả': 0


24/10/10 09:59:24 INFO MongoClientCache: Closing MongoClient: [localhost:27017]
24/10/10 09:59:24 INFO connection: Closed connection [connectionId{localValue:120, serverValue:412}] to localhost:27017 because the pool has been closed.
24/10/10 09:59:24 INFO Executor: Finished task 0.0 in stage 40.0 (TID 31). 1900 bytes result sent to driver
24/10/10 09:59:24 INFO TaskSetManager: Finished task 0.0 in stage 40.0 (TID 31) in 86 ms on 192.168.34.128 (executor driver) (1/1)
24/10/10 09:59:24 INFO TaskSchedulerImpl: Removed TaskSet 40.0, whose tasks have all completed, from pool 
24/10/10 09:59:24 INFO DAGScheduler: ShuffleMapStage 40 (count at NativeMethodAccessorImpl.java:0) finished in 0,089 s
24/10/10 09:59:24 INFO DAGScheduler: looking for newly runnable stages
24/10/10 09:59:24 INFO DAGScheduler: running: Set()
24/10/10 09:59:24 INFO DAGScheduler: waiting: Set()
24/10/10 09:59:24 INFO DAGScheduler: failed: Set()
24/10/10 09:59:24 INFO SparkContext: Starting job: count at NativeMethodAcc

...

24/10/10 09:59:24 INFO connection: Opened connection [connectionId{localValue:138, serverValue:430}] to localhost:27017
24/10/10 09:59:24 INFO MongoClientCache: Closing MongoClient: [localhost:27017]
24/10/10 09:59:24 INFO connection: Closed connection [connectionId{localValue:138, serverValue:430}] to localhost:27017 because the pool has been closed.
24/10/10 09:59:24 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 09:59:24 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 09:59:24 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 09:59:24 INFO connection: Opened connection [connectionId{localValue:139, serverValue:431}] to localhost:27017
24/10/10 09:59:24 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECT

Số giá trị null trong cột 'Số_phòng_ngủ': 142
Số giá trị null trong cột 'Số_phòng_toilet': 195
Số giá trị null trong cột 'Thành_phố': 0


24/10/10 09:59:24 INFO SparkContext: Created broadcast 46 from broadcast at DAGScheduler.scala:1585
24/10/10 09:59:24 INFO DAGScheduler: Submitting 1 missing tasks from ResultStage 57 (MapPartitionsRDD[196] at count at NativeMethodAccessorImpl.java:0) (first 15 tasks are for partitions Vector(0))
24/10/10 09:59:24 INFO TaskSchedulerImpl: Adding task set 57.0 with 1 tasks resource profile 0
24/10/10 09:59:24 INFO TaskSetManager: Starting task 0.0 in stage 57.0 (TID 42) (192.168.34.128, executor driver, partition 0, NODE_LOCAL, 10236 bytes) 
24/10/10 09:59:24 INFO Executor: Running task 0.0 in stage 57.0 (TID 42)
24/10/10 09:59:24 INFO ShuffleBlockFetcherIterator: Getting 1 (60.0 B) non-empty blocks including 1 (60.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) push-merged-local and 0 (0.0 B) remote blocks
24/10/10 09:59:24 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/10/10 09:59:24 INFO Executor: Finished task 0.0 in stage 57.0 (TID 42). 3995 bytes result sen

Số giá trị null trong cột 'Tiêu_đề': 0
Số giá trị null trong cột 'Đường': 0


24/10/10 09:59:24 INFO MongoRelation: requiredColumns: Đường, filters: IsNull(Đường)
24/10/10 09:59:25 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 09:59:25 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 09:59:25 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 09:59:25 INFO connection: Opened connection [connectionId{localValue:174, serverValue:466}] to localhost:27017
24/10/10 09:59:25 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireVersion=17, maxDocumentSize=16777216, logicalSessionTimeoutMinutes=30, roundTripTimeNanos=230167}
24/10/10 09:59:25 INFO connection: Opened connection [connectionId{localValue:175, serverValue:467}] to localhost:27017
24/10/10 09:5

In [25]:
num_rows = df.count()
num_columns = len(df.columns)
print(f"Kích thước của DataFrame: {num_rows} hàng và {num_columns} cột")


Kích thước của DataFrame: 1056 hàng và 12 cột


24/10/10 09:59:52 INFO CodeGenerator: Code generated in 3.216333 ms
24/10/10 09:59:52 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 09:59:52 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 09:59:52 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 09:59:52 INFO connection: Opened connection [connectionId{localValue:183, serverValue:475}] to localhost:27017
24/10/10 09:59:52 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireVersion=17, maxDocumentSize=16777216, logicalSessionTimeoutMinutes=30, roundTripTimeNanos=213459}
24/10/10 09:59:52 INFO connection: Opened connection [connectionId{localValue:184, serverValue:476}] to localhost:27017
24/10/10 09:59:52 INFO MongoCl

In [26]:
# Xử Lý giá trị null cột Diện_tích
# Thay thế NULL bằng giá trị trung vị
median_area = df.approxQuantile("Diện_tích", [0.5], 0)[0]
df = df.fillna({"Diện_tích": median_area})

df.show()

24/10/10 10:00:16 INFO MongoRelation: requiredColumns: Diện_tích, filters: 
24/10/10 10:00:16 INFO CodeGenerator: Code generated in 6.920041 ms
24/10/10 10:00:16 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 10:00:16 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 10:00:16 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 10:00:16 INFO connection: Opened connection [connectionId{localValue:190, serverValue:482}] to localhost:27017
24/10/10 10:00:16 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireVersion=17, maxDocumentSize=16777216, logicalSessionTimeoutMinutes=30, roundTripTimeNanos=211583}
24/10/10 10:00:16 INFO connection: Opened connection [connectionId{localVa

+---------+-----------+------+--------------------+----------+-------+-----------+------------+---------------+-----------+--------------------+-------------------+
|Diện_tích|        Giá|Mã_tin|               Mô_tả| Ngày_đăng|Pháp_lý|       Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|              Đường|
+---------+-----------+------+--------------------+----------+-------+-----------+------------+---------------+-----------+--------------------+-------------------+
|     80.0| 4700000000|121248|       Thông tin nhà|2024-10-06|   NULL|    Tân Phú|           2|              2|Hồ Chí Minh|Bán nhà chính chủ...|          Phú Trung|
|     70.0| 5800000000|121204|Nhà  Nguyễn Văn Đ...|2024-10-02|  Sổ đỏ| Bình Thạnh|           2|              2|Hồ Chí Minh|Nhà  Nguyễn Văn Đ...|     Nguyễn Văn Đậu|
|     45.0| 9800000000|121156|BÁN NHÀ NGUYỄN KH...|2024-10-02|  Sổ đỏ|   Cầu Giấy|           5|              3|     Hà Nội|BÁN NHÀ NGUYỄN KH...|  Nguyễn Khánh Toàn|
|     55.0

24/10/10 10:00:16 INFO MongoRelation: requiredColumns: Diện_tích, Giá, Mã_tin, Mô_tả, Ngày_đăng, Pháp_lý, Quận, Số_phòng_ngủ, Số_phòng_toilet, Thành_phố, Tiêu_đề, Đường, filters: 
24/10/10 10:00:16 INFO CodeGenerator: Code generated in 14.407708 ms
24/10/10 10:00:16 INFO CodeGenerator: Code generated in 2.523833 ms
24/10/10 10:00:16 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 10:00:16 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 10:00:16 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 10:00:16 INFO connection: Opened connection [connectionId{localValue:197, serverValue:489}] to localhost:27017
24/10/10 10:00:16 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWir

In [27]:
#xử lý giá trị null của Số_phòng_ngủ và Số_phòng_toilet bằng 0
df = df.fillna({"Số_phòng_ngủ": 0, "Số_phòng_toilet": 0})
df.show()


+---------+-----------+------+--------------------+----------+-------+-----------+------------+---------------+-----------+--------------------+-------------------+
|Diện_tích|        Giá|Mã_tin|               Mô_tả| Ngày_đăng|Pháp_lý|       Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|              Đường|
+---------+-----------+------+--------------------+----------+-------+-----------+------------+---------------+-----------+--------------------+-------------------+
|     80.0| 4700000000|121248|       Thông tin nhà|2024-10-06|   NULL|    Tân Phú|           2|              2|Hồ Chí Minh|Bán nhà chính chủ...|          Phú Trung|
|     70.0| 5800000000|121204|Nhà  Nguyễn Văn Đ...|2024-10-02|  Sổ đỏ| Bình Thạnh|           2|              2|Hồ Chí Minh|Nhà  Nguyễn Văn Đ...|     Nguyễn Văn Đậu|
|     45.0| 9800000000|121156|BÁN NHÀ NGUYỄN KH...|2024-10-02|  Sổ đỏ|   Cầu Giấy|           5|              3|     Hà Nội|BÁN NHÀ NGUYỄN KH...|  Nguyễn Khánh Toàn|
|     55.0

24/10/10 10:00:32 INFO MongoRelation: requiredColumns: Diện_tích, Giá, Mã_tin, Mô_tả, Ngày_đăng, Pháp_lý, Quận, Số_phòng_ngủ, Số_phòng_toilet, Thành_phố, Tiêu_đề, Đường, filters: 
24/10/10 10:00:32 INFO CodeGenerator: Code generated in 9.902667 ms
24/10/10 10:00:32 INFO CodeGenerator: Code generated in 2.654583 ms
24/10/10 10:00:32 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 10:00:32 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 10:00:32 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 10:00:32 INFO connection: Opened connection [connectionId{localValue:204, serverValue:496}] to localhost:27017
24/10/10 10:00:32 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWire

In [28]:
#Thay thế giá trị null của Pháp_lý thành "Unknown"
df = df.fillna({"Pháp_lý": "Unknown"})
df.show()


+---------+-----------+------+--------------------+----------+-------+-----------+------------+---------------+-----------+--------------------+-------------------+
|Diện_tích|        Giá|Mã_tin|               Mô_tả| Ngày_đăng|Pháp_lý|       Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|              Đường|
+---------+-----------+------+--------------------+----------+-------+-----------+------------+---------------+-----------+--------------------+-------------------+
|     80.0| 4700000000|121248|       Thông tin nhà|2024-10-06|Unknown|    Tân Phú|           2|              2|Hồ Chí Minh|Bán nhà chính chủ...|          Phú Trung|
|     70.0| 5800000000|121204|Nhà  Nguyễn Văn Đ...|2024-10-02|  Sổ đỏ| Bình Thạnh|           2|              2|Hồ Chí Minh|Nhà  Nguyễn Văn Đ...|     Nguyễn Văn Đậu|
|     45.0| 9800000000|121156|BÁN NHÀ NGUYỄN KH...|2024-10-02|  Sổ đỏ|   Cầu Giấy|           5|              3|     Hà Nội|BÁN NHÀ NGUYỄN KH...|  Nguyễn Khánh Toàn|
|     55.0

24/10/10 10:00:44 INFO MongoRelation: requiredColumns: Diện_tích, Giá, Mã_tin, Mô_tả, Ngày_đăng, Pháp_lý, Quận, Số_phòng_ngủ, Số_phòng_toilet, Thành_phố, Tiêu_đề, Đường, filters: 
24/10/10 10:00:44 INFO CodeGenerator: Code generated in 10.460333 ms
24/10/10 10:00:44 INFO CodeGenerator: Code generated in 3.090208 ms
24/10/10 10:00:44 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 10:00:44 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 10:00:44 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 10:00:44 INFO connection: Opened connection [connectionId{localValue:211, serverValue:503}] to localhost:27017
24/10/10 10:00:44 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWir

In [29]:
#kiểm tra lại 
# Tạo danh sách các cột kiểm tra giá trị null để dễ dàng xử lý
columns_to_check = ['Diện_tích', 'Giá', 'Mã_tin', 'Mô_tả', 'Ngày_đăng', 'Pháp_lý', 'Quận', 'Số_phòng_ngủ', 'Số_phòng_toilet', 'Thành_phố', 'Tiêu_đề', 'Đường']

# Lặp qua từng cột và đếm số giá trị null
for col in columns_to_check:
    null_count = df.filter(F.col(col).isNull()).count()  # Đếm số giá trị null trong cột
    print(f"Số giá trị null trong cột '{col}': {null_count}")


24/10/10 10:00:57 INFO CodeGenerator: Code generated in 4.642292 ms
24/10/10 10:00:57 INFO SparkContext: Starting job: count at <unknown>:0
24/10/10 10:00:57 INFO DAGScheduler: Registering RDD 263 (count at <unknown>:0) as input to shuffle 19
24/10/10 10:00:57 INFO DAGScheduler: Got job 55 (count at <unknown>:0) with 1 output partitions
24/10/10 10:00:57 INFO DAGScheduler: Final stage: ResultStage 75 (count at <unknown>:0)
24/10/10 10:00:57 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 74)
24/10/10 10:00:57 INFO DAGScheduler: Missing parents: List()
24/10/10 10:00:57 INFO DAGScheduler: Submitting ResultStage 75 (MapPartitionsRDD[266] at count at <unknown>:0), which has no missing parents
24/10/10 10:00:57 INFO MemoryStore: Block broadcast_59 stored as values in memory (estimated size 12.5 KiB, free 434.1 MiB)
24/10/10 10:00:57 INFO MemoryStore: Block broadcast_59_piece0 stored as bytes in memory (estimated size 5.9 KiB, free 434.1 MiB)
24/10/10 10:00:57 INFO BlockMana

Số giá trị null trong cột 'Diện_tích': 0
Số giá trị null trong cột 'Giá': 0


24/10/10 10:00:58 INFO connection: Opened connection [connectionId{localValue:227, serverValue:519}] to localhost:27017
24/10/10 10:00:58 INFO MongoClientCache: Closing MongoClient: [localhost:27017]
24/10/10 10:00:58 INFO connection: Closed connection [connectionId{localValue:227, serverValue:519}] to localhost:27017 because the pool has been closed.
24/10/10 10:00:58 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 10:00:58 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 10:00:58 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 10:00:58 INFO connection: Opened connection [connectionId{localValue:228, serverValue:520}] to localhost:27017
24/10/10 10:00:58 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECT

Số giá trị null trong cột 'Mã_tin': 0
Số giá trị null trong cột 'Mô_tả': 0


24/10/10 10:00:58 INFO MongoClientCache: Closing MongoClient: [localhost:27017]
24/10/10 10:00:58 INFO connection: Closed connection [connectionId{localValue:239, serverValue:531}] to localhost:27017 because the pool has been closed.
24/10/10 10:00:58 INFO Executor: Finished task 0.0 in stage 82.0 (TID 60). 1900 bytes result sent to driver
24/10/10 10:00:58 INFO TaskSetManager: Finished task 0.0 in stage 82.0 (TID 60) in 77 ms on 192.168.34.128 (executor driver) (1/1)
24/10/10 10:00:58 INFO TaskSchedulerImpl: Removed TaskSet 82.0, whose tasks have all completed, from pool 
24/10/10 10:00:58 INFO DAGScheduler: ShuffleMapStage 82 (count at <unknown>:0) finished in 0,079 s
24/10/10 10:00:58 INFO DAGScheduler: looking for newly runnable stages
24/10/10 10:00:58 INFO DAGScheduler: running: Set()
24/10/10 10:00:58 INFO DAGScheduler: waiting: Set()
24/10/10 10:00:58 INFO DAGScheduler: failed: Set()
24/10/10 10:00:58 INFO SparkContext: Starting job: count at <unknown>:0
24/10/10 10:00:58 INFO 

Số giá trị null trong cột 'Ngày_đăng': 0
Số giá trị null trong cột 'Pháp_lý': 0
Số giá trị null trong cột 'Quận': 0
Số giá trị null trong cột 'Số_phòng_ngủ': 0
Số giá trị null trong cột 'Số_phòng_toilet': 0


24/10/10 10:00:58 INFO BlockManagerInfo: Removed broadcast_62_piece0 on 192.168.34.128:59436 in memory (size: 13.1 KiB, free: 434.3 MiB)
24/10/10 10:00:58 INFO BlockManagerInfo: Removed broadcast_60_piece0 on 192.168.34.128:59436 in memory (size: 17.1 KiB, free: 434.3 MiB)
24/10/10 10:00:58 INFO BlockManagerInfo: Removed broadcast_67_piece0 on 192.168.34.128:59436 in memory (size: 5.9 KiB, free: 434.3 MiB)
24/10/10 10:00:58 INFO BlockManagerInfo: Removed broadcast_66_piece0 on 192.168.34.128:59436 in memory (size: 13.9 KiB, free: 434.3 MiB)
24/10/10 10:00:58 INFO BlockManagerInfo: Removed broadcast_63_piece0 on 192.168.34.128:59436 in memory (size: 5.9 KiB, free: 434.3 MiB)
24/10/10 10:00:58 INFO BlockManagerInfo: Removed broadcast_69_piece0 on 192.168.34.128:59436 in memory (size: 13.1 KiB, free: 434.4 MiB)
24/10/10 10:00:58 INFO BlockManagerInfo: Removed broadcast_64_piece0 on 192.168.34.128:59436 in memory (size: 14.6 KiB, free: 434.4 MiB)
24/10/10 10:00:58 INFO BlockManagerInfo: Re

Số giá trị null trong cột 'Thành_phố': 0
Số giá trị null trong cột 'Tiêu_đề': 0
Số giá trị null trong cột 'Đường': 0


24/10/10 10:00:58 INFO MongoClientCache: Closing MongoClient: [localhost:27017]
24/10/10 10:00:58 INFO connection: Closed connection [connectionId{localValue:271, serverValue:563}] to localhost:27017 because the pool has been closed.
24/10/10 10:00:58 INFO Executor: Finished task 0.0 in stage 100.0 (TID 71). 1900 bytes result sent to driver
24/10/10 10:00:58 INFO TaskSetManager: Finished task 0.0 in stage 100.0 (TID 71) in 73 ms on 192.168.34.128 (executor driver) (1/1)
24/10/10 10:00:58 INFO TaskSchedulerImpl: Removed TaskSet 100.0, whose tasks have all completed, from pool 
24/10/10 10:00:58 INFO DAGScheduler: ShuffleMapStage 100 (count at <unknown>:0) finished in 0,075 s
24/10/10 10:00:58 INFO DAGScheduler: looking for newly runnable stages
24/10/10 10:00:58 INFO DAGScheduler: running: Set()
24/10/10 10:00:58 INFO DAGScheduler: waiting: Set()
24/10/10 10:00:58 INFO DAGScheduler: failed: Set()
24/10/10 10:00:58 INFO SparkContext: Starting job: count at <unknown>:0
24/10/10 10:00:58 I

In [30]:
# Đếm số giá trị bằng 0 trong cột Giá
zero_count = df.filter(F.col("Giá") == 0).count()
# In ra số lượng giá trị bằng 0
print(f"Số giá trị bằng 0 trong cột 'Giá': {zero_count}")


Số giá trị bằng 0 trong cột 'Giá': 6


24/10/10 10:01:17 INFO MongoRelation: requiredColumns: Giá, filters: 
24/10/10 10:01:17 INFO CodeGenerator: Code generated in 3.732167 ms
24/10/10 10:01:17 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 10:01:17 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 10:01:17 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 10:01:17 INFO connection: Opened connection [connectionId{localValue:281, serverValue:573}] to localhost:27017
24/10/10 10:01:17 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireVersion=17, maxDocumentSize=16777216, logicalSessionTimeoutMinutes=30, roundTripTimeNanos=282167}
24/10/10 10:01:17 INFO connection: Opened connection [connectionId{localValue:28

In [31]:
# nhận thấy có 5 giá trị của cột Giá=0 nên xóa đi
# Xóa các dòng có giá trị bằng 0 trong cột 'Giá'
df = df.filter(F.col("Giá") != 0)

# Hiển thị DataFrame sau khi xóa
df.show()


+---------+-----------+------+--------------------+----------+-------+-----------+------------+---------------+-----------+--------------------+-------------------+
|Diện_tích|        Giá|Mã_tin|               Mô_tả| Ngày_đăng|Pháp_lý|       Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|              Đường|
+---------+-----------+------+--------------------+----------+-------+-----------+------------+---------------+-----------+--------------------+-------------------+
|     80.0| 4700000000|121248|       Thông tin nhà|2024-10-06|Unknown|    Tân Phú|           2|              2|Hồ Chí Minh|Bán nhà chính chủ...|          Phú Trung|
|     70.0| 5800000000|121204|Nhà  Nguyễn Văn Đ...|2024-10-02|  Sổ đỏ| Bình Thạnh|           2|              2|Hồ Chí Minh|Nhà  Nguyễn Văn Đ...|     Nguyễn Văn Đậu|
|     45.0| 9800000000|121156|BÁN NHÀ NGUYỄN KH...|2024-10-02|  Sổ đỏ|   Cầu Giấy|           5|              3|     Hà Nội|BÁN NHÀ NGUYỄN KH...|  Nguyễn Khánh Toàn|
|     55.0

24/10/10 10:01:37 INFO MongoRelation: requiredColumns: Diện_tích, Giá, Mã_tin, Mô_tả, Ngày_đăng, Pháp_lý, Quận, Số_phòng_ngủ, Số_phòng_toilet, Thành_phố, Tiêu_đề, Đường, filters: 
24/10/10 10:01:37 INFO BlockManagerInfo: Removed broadcast_76_piece0 on 192.168.34.128:59436 in memory (size: 5.9 KiB, free: 434.3 MiB)
24/10/10 10:01:37 INFO CodeGenerator: Code generated in 20.2305 ms
24/10/10 10:01:37 INFO BlockManagerInfo: Removed broadcast_79_piece0 on 192.168.34.128:59436 in memory (size: 17.3 KiB, free: 434.3 MiB)
24/10/10 10:01:37 INFO BlockManagerInfo: Removed broadcast_72_piece0 on 192.168.34.128:59436 in memory (size: 5.9 KiB, free: 434.3 MiB)
24/10/10 10:01:37 INFO BlockManagerInfo: Removed broadcast_78_piece0 on 192.168.34.128:59436 in memory (size: 5.9 KiB, free: 434.3 MiB)
24/10/10 10:01:37 INFO BlockManagerInfo: Removed broadcast_75_piece0 on 192.168.34.128:59436 in memory (size: 14.6 KiB, free: 434.4 MiB)
24/10/10 10:01:37 INFO BlockManagerInfo: Removed broadcast_74_piece0 on

In [32]:
#xem lại kích thước
num_rows = df.count()
num_columns = len(df.columns)
print(f"Kích thước của DataFrame: {num_rows} hàng và {num_columns} cột")


Kích thước của DataFrame: 1050 hàng và 12 cột


24/10/10 10:01:53 INFO MongoRelation: requiredColumns: Giá, filters: 
24/10/10 10:01:53 INFO CodeGenerator: Code generated in 4.420083 ms
24/10/10 10:01:53 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 10:01:53 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 10:01:53 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 10:01:53 INFO connection: Opened connection [connectionId{localValue:295, serverValue:587}] to localhost:27017
24/10/10 10:01:53 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireVersion=17, maxDocumentSize=16777216, logicalSessionTimeoutMinutes=30, roundTripTimeNanos=259958}
24/10/10 10:01:53 INFO connection: Opened connection [connectionId{localValue:29

In [33]:
# đổi tên cột
df = df \
    .withColumnRenamed("Diện_tích", "Area") \
    .withColumnRenamed("Giá", "Price") \
    .withColumnRenamed("Mô_tả", "Description") \
    .withColumnRenamed("Ngày_đăng", "Posted_Date") \
    .withColumnRenamed("Pháp_lý", "Legal_Status") \
    .withColumnRenamed("Quận", "District") \
    .withColumnRenamed("Số_phòng_ngủ", "Bedrooms") \
    .withColumnRenamed("Số_phòng_toilet", "Bathrooms") \
    .withColumnRenamed("Thành_phố", "City") \
    .withColumnRenamed("Tiêu_đề", "Title") \
    .withColumnRenamed("Đường", "Street_Name")

# Hiển thị dữ liệu sau khi đổi tên cột
df.show()


+-----+-----------+------+--------------------+-----------+------------+-----------+--------+---------+-----------+--------------------+-------------------+
| Area|      Price|Mã_tin|         Description|Posted_Date|Legal_Status|   District|Bedrooms|Bathrooms|       City|               Title|        Street_Name|
+-----+-----------+------+--------------------+-----------+------------+-----------+--------+---------+-----------+--------------------+-------------------+
| 80.0| 4700000000|121248|       Thông tin nhà| 2024-10-06|     Unknown|    Tân Phú|       2|        2|Hồ Chí Minh|Bán nhà chính chủ...|          Phú Trung|
| 70.0| 5800000000|121204|Nhà  Nguyễn Văn Đ...| 2024-10-02|       Sổ đỏ| Bình Thạnh|       2|        2|Hồ Chí Minh|Nhà  Nguyễn Văn Đ...|     Nguyễn Văn Đậu|
| 45.0| 9800000000|121156|BÁN NHÀ NGUYỄN KH...| 2024-10-02|       Sổ đỏ|   Cầu Giấy|       5|        3|     Hà Nội|BÁN NHÀ NGUYỄN KH...|  Nguyễn Khánh Toàn|
| 55.0| 6800000000|121121|Cơ hội vàng,mặt p...| 2024-10-02

24/10/10 10:02:10 INFO MongoRelation: requiredColumns: Diện_tích, Giá, Mã_tin, Mô_tả, Ngày_đăng, Pháp_lý, Quận, Số_phòng_ngủ, Số_phòng_toilet, Thành_phố, Tiêu_đề, Đường, filters: 
24/10/10 10:02:10 INFO CodeGenerator: Code generated in 3.952875 ms
24/10/10 10:02:10 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 10:02:10 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 10:02:10 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 10:02:10 INFO connection: Opened connection [connectionId{localValue:302, serverValue:594}] to localhost:27017
24/10/10 10:02:10 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireVersion=17, maxDocumentSize=16777216, logicalSessionTimeoutMinutes=3

In [38]:
df.printSchema()

# Hiển thị dữ liệu sau khi xử lý
df.show()

root
 |-- Area: float (nullable = false)
 |-- Price: long (nullable = true)
 |-- Mã_tin: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Posted_Date: date (nullable = true)
 |-- Legal_Status: string (nullable = false)
 |-- District: string (nullable = true)
 |-- Bedrooms: integer (nullable = false)
 |-- Bathrooms: integer (nullable = false)
 |-- City: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Street_Name: string (nullable = true)

+-----+-----------+------+--------------------+-----------+------------+-----------+--------+---------+-----------+--------------------+-------------------+
| Area|      Price|Mã_tin|         Description|Posted_Date|Legal_Status|   District|Bedrooms|Bathrooms|       City|               Title|        Street_Name|
+-----+-----------+------+--------------------+-----------+------------+-----------+--------+---------+-----------+--------------------+-------------------+
| 80.0| 4700000000|121248|       Thông tin n

24/10/10 10:08:46 INFO MongoRelation: requiredColumns: Diện_tích, Giá, Mã_tin, Mô_tả, Ngày_đăng, Pháp_lý, Quận, Số_phòng_ngủ, Số_phòng_toilet, Thành_phố, Tiêu_đề, Đường, filters: 
24/10/10 10:08:46 INFO CodeGenerator: Code generated in 5.115417 ms
24/10/10 10:08:46 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 10:08:46 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 10:08:46 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 10:08:46 INFO connection: Opened connection [connectionId{localValue:330, serverValue:622}] to localhost:27017
24/10/10 10:08:46 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireVersion=17, maxDocumentSize=16777216, logicalSessionTimeoutMinutes=3

24/10/10 10:14:03 INFO BlockManagerInfo: Removed broadcast_87_piece0 on 192.168.34.128:59436 in memory (size: 21.1 KiB, free: 434.2 MiB)
24/10/10 10:14:03 INFO BlockManagerInfo: Removed broadcast_88_piece0 on 192.168.34.128:59436 in memory (size: 19.2 KiB, free: 434.3 MiB)
24/10/10 10:14:03 INFO BlockManagerInfo: Removed broadcast_90_piece0 on 192.168.34.128:59436 in memory (size: 102.3 KiB, free: 434.4 MiB)
24/10/10 10:14:03 INFO BlockManagerInfo: Removed broadcast_91_piece0 on 192.168.34.128:59436 in memory (size: 7.1 KiB, free: 434.4 MiB)
24/10/10 10:14:03 INFO BlockManagerInfo: Removed broadcast_92_piece0 on 192.168.34.128:59436 in memory (size: 26.9 KiB, free: 434.4 MiB)
24/10/10 10:14:03 INFO BlockManagerInfo: Removed broadcast_89_piece0 on 192.168.34.128:59436 in memory (size: 4.2 KiB, free: 434.4 MiB)


In [42]:
# Bảng Property (Bất động sản)
property_df = df.select(
    "Mã_tin",
    "Area",
    "Price",
    "Bedrooms",
    "Bathrooms",
    "Legal_Status",
    "Description",
    "Posted_Date"
).withColumnRenamed("Mã_tin", "Property_ID")  # Đổi tên để tạo khóa chính

# Bảng Location (Vị trí)
location_df = df.select(
    F.monotonically_increasing_id().alias("Location_ID"),
    "District",
    "City",
    "Street_Name"
).distinct()

# Bảng Listing (Danh sách tin)
listing_df = df.select(
    F.monotonically_increasing_id().alias("Listing_ID"),
    "Mã_tin",
    F.monotonically_increasing_id().alias("Location_ID"),
    "Title"
).withColumnRenamed("Mã_tin", "Property_ID")  # Tạo khóa ngoại Property_ID

# Hiển thị các bảng đã chia
print("Property Table:")
property_df.show(2)

print("Location Table:")
location_df.show(2)

print("Listing Table:")
listing_df.show(2)

# # Cấu hình để kết nối với PostgreSQL
# postgres_url = "jdbc:postgresql://localhost:5432/your_database"
# properties = {
#     "user": "your_username",
#     "password": "your_password",
#     "driver": "org.postgresql.Driver"
# }

# # Ghi các bảng vào PostgreSQL với khóa chính và khóa ngoại
# property_df.write.jdbc(postgres_url, "property", mode="overwrite", properties=properties)
# location_df.write.jdbc(postgres_url, "location", mode="overwrite", properties=properties)
# listing_df.write.jdbc(postgres_url, "listing", mode="overwrite", properties=properties)

# # Trong PostgreSQL:
# # Bảng `property` sẽ có khóa chính là `Property_ID`.
# # Bảng `location` sẽ có khóa chính là `Location_ID`.
# # Bảng `listing` sẽ có khóa ngoại `Property_ID` liên kết với bảng `property`
# # và `Location_ID` liên kết với bảng `location`.


Property Table:
+-----------+----+----------+--------+---------+------------+--------------------+-----------+
|Property_ID|Area|     Price|Bedrooms|Bathrooms|Legal_Status|         Description|Posted_Date|
+-----------+----+----------+--------+---------+------------+--------------------+-----------+
|     121248|80.0|4700000000|       2|        2|     Unknown|       Thông tin nhà| 2024-10-06|
|     121204|70.0|5800000000|       2|        2|       Sổ đỏ|Nhà  Nguyễn Văn Đ...| 2024-10-02|
+-----------+----+----------+--------+---------+------------+--------------------+-----------+
only showing top 2 rows

Location Table:


24/10/10 10:44:36 INFO MongoRelation: requiredColumns: Diện_tích, Giá, Mã_tin, Mô_tả, Ngày_đăng, Pháp_lý, Số_phòng_ngủ, Số_phòng_toilet, filters: 
24/10/10 10:44:36 INFO CodeGenerator: Code generated in 2.951959 ms
24/10/10 10:44:36 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 10:44:36 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 10:44:36 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 10:44:36 INFO connection: Opened connection [connectionId{localValue:358, serverValue:650}] to localhost:27017
24/10/10 10:44:36 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireVersion=17, maxDocumentSize=16777216, logicalSessionTimeoutMinutes=30, roundTripTimeNanos=171167}
24/

+-----------+--------+-----------+-------------+
|Location_ID|District|       City|  Street_Name|
+-----------+--------+-----------+-------------+
|         86| Quận 10|Hồ Chí Minh|Lê Hồng Phong|
|        286|  Quận 3|Hồ Chí Minh|    Lê Văn Sỹ|
+-----------+--------+-----------+-------------+
only showing top 2 rows

Listing Table:
+----------+-----------+-----------+--------------------+
|Listing_ID|Property_ID|Location_ID|               Title|
+----------+-----------+-----------+--------------------+
|         0|     121248|          0|Bán nhà chính chủ...|
|         1|     121204|          1|Nhà  Nguyễn Văn Đ...|
+----------+-----------+-----------+--------------------+
only showing top 2 rows



24/10/10 10:44:36 INFO MongoClientCache: Closing MongoClient: [localhost:27017]
24/10/10 10:44:36 INFO connection: Closed connection [connectionId{localValue:371, serverValue:663}] to localhost:27017 because the pool has been closed.
24/10/10 10:44:36 INFO Executor: Finished task 0.0 in stage 129.0 (TID 93). 3275 bytes result sent to driver
24/10/10 10:44:36 INFO TaskSetManager: Finished task 0.0 in stage 129.0 (TID 93) in 103 ms on 192.168.34.128 (executor driver) (1/1)
24/10/10 10:44:36 INFO TaskSchedulerImpl: Removed TaskSet 129.0, whose tasks have all completed, from pool 
24/10/10 10:44:36 INFO DAGScheduler: ShuffleMapStage 129 (showString at <unknown>:0) finished in 0,105 s
24/10/10 10:44:36 INFO DAGScheduler: looking for newly runnable stages
24/10/10 10:44:36 INFO DAGScheduler: running: Set()
24/10/10 10:44:36 INFO DAGScheduler: waiting: Set()
24/10/10 10:44:36 INFO DAGScheduler: failed: Set()
24/10/10 10:44:36 INFO ShufflePartitionsUtil: For shuffle(36), advisory target size: 

In [43]:
# Kết nối tới PostgreSQL

jdbc_url = "jdbc:postgresql://localhost:5432/database_bds"

connection_properties = {
    "user": "vantruong179",
    "password": "trong1709",
    "driver": "org.postgresql.Driver"
}


In [53]:
#hàm ghi vô postgres
def write_to_postgres(df, table_name):
    try:
        df.write.jdbc(
            url=jdbc_url,
            table=table_name,
            mode="overwrite",
            properties=connection_properties
        )
        print(f"Đã ghi thành công bảng {table_name} vào PostgreSQL!")
    except Exception as e:
        print(f"Lỗi khi ghi bảng {table_name} vào PostgreSQL: {e}")
#Gọi lại hàm để lưu bảng vào PostgreSQL

# # Ghi các bảng vào PostgreSQL với khóa chính và khóa ngoại
# property_df.write.jdbc(postgres_url, "property", mode="overwrite", properties=properties)
# location_df.write.jdbc(postgres_url, "location", mode="overwrite", properties=properties)
# listing_df.write.jdbc(postgres_url, "listing", mode="overwrite", properties=properties)

write_to_postgres(property_df, "property")
write_to_postgres(location_df, "location")
write_to_postgres(listing_df, "listing")

#them bang de test
test_df = df.select('Title','Mã_tin','Area','Price','City','Bedrooms','Legal_Status','Bathrooms','District','Street_Name','Description','Posted_Date').withColumnRenamed("Mã_tin", "Property_ID")
write_to_postgres(test_df, "table_test")

24/10/10 14:32:12 INFO MongoRelation: requiredColumns: Diện_tích, Giá, Mã_tin, Mô_tả, Ngày_đăng, Pháp_lý, Số_phòng_ngủ, Số_phòng_toilet, filters: 
24/10/10 14:32:12 INFO cluster: Cluster created with settings {hosts=[localhost:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
24/10/10 14:32:12 INFO MongoClientCache: Creating MongoClient: [localhost:27017]
24/10/10 14:32:12 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
24/10/10 14:32:12 INFO connection: Opened connection [connectionId{localValue:511, serverValue:803}] to localhost:27017
24/10/10 14:32:12 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=localhost:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireVersion=17, maxDocumentSize=16777216, logicalSessionTimeoutMinutes=30, roundTripTimeNanos=255459}
24/10/10 14:32:12 INFO connection: Opened connection [connectionId{loca

Đã ghi thành công bảng property vào PostgreSQL!


24/10/10 14:32:13 INFO BlockManagerInfo: Removed broadcast_119_piece0 on 192.168.34.128:59436 in memory (size: 24.0 KiB, free: 434.4 MiB)
24/10/10 14:32:13 INFO Executor: Finished task 0.0 in stage 163.0 (TID 121). 5559 bytes result sent to driver
24/10/10 14:32:13 INFO TaskSetManager: Finished task 0.0 in stage 163.0 (TID 121) in 86 ms on 192.168.34.128 (executor driver) (1/1)
24/10/10 14:32:13 INFO TaskSchedulerImpl: Removed TaskSet 163.0, whose tasks have all completed, from pool 
24/10/10 14:32:13 INFO DAGScheduler: ResultStage 163 (jdbc at <unknown>:0) finished in 0,091 s
24/10/10 14:32:13 INFO DAGScheduler: Job 121 is finished. Cancelling potential speculative or zombie tasks for this job
24/10/10 14:32:13 INFO TaskSchedulerImpl: Killing all running tasks in stage 163: Stage finished
24/10/10 14:32:13 INFO DAGScheduler: Job 121 finished: jdbc at <unknown>:0, took 0,093076 s
24/10/10 14:32:13 INFO MongoRelation: requiredColumns: Giá, Mã_tin, Tiêu_đề, filters: 
24/10/10 14:32:13 IN

Đã ghi thành công bảng location vào PostgreSQL!


24/10/10 14:32:13 INFO PythonUDFRunner: Times: total = 35, boot = -496, init = 529, finish = 2
24/10/10 14:32:13 INFO MongoClientCache: Closing MongoClient: [localhost:27017]
24/10/10 14:32:13 INFO connection: Closed connection [connectionId{localValue:531, serverValue:823}] to localhost:27017 because the pool has been closed.
24/10/10 14:32:13 INFO Executor: Finished task 0.0 in stage 164.0 (TID 122). 2014 bytes result sent to driver
24/10/10 14:32:13 INFO TaskSetManager: Finished task 0.0 in stage 164.0 (TID 122) in 157 ms on 192.168.34.128 (executor driver) (1/1)
24/10/10 14:32:13 INFO TaskSchedulerImpl: Removed TaskSet 164.0, whose tasks have all completed, from pool 
24/10/10 14:32:13 INFO DAGScheduler: ResultStage 164 (jdbc at <unknown>:0) finished in 0,159 s
24/10/10 14:32:13 INFO DAGScheduler: Job 122 is finished. Cancelling potential speculative or zombie tasks for this job
24/10/10 14:32:13 INFO TaskSchedulerImpl: Killing all running tasks in stage 164: Stage finished
24/10/1

Đã ghi thành công bảng listing vào PostgreSQL!
Đã ghi thành công bảng table_test vào PostgreSQL!


24/10/10 14:32:13 INFO PythonUDFRunner: Times: total = 39, boot = -208, init = 245, finish = 2
24/10/10 14:32:13 INFO MongoClientCache: Closing MongoClient: [localhost:27017]
24/10/10 14:32:13 INFO connection: Closed connection [connectionId{localValue:537, serverValue:829}] to localhost:27017 because the pool has been closed.
24/10/10 14:32:13 INFO Executor: Finished task 0.0 in stage 165.0 (TID 123). 2238 bytes result sent to driver
24/10/10 14:32:13 INFO TaskSetManager: Finished task 0.0 in stage 165.0 (TID 123) in 245 ms on 192.168.34.128 (executor driver) (1/1)
24/10/10 14:32:13 INFO TaskSchedulerImpl: Removed TaskSet 165.0, whose tasks have all completed, from pool 
24/10/10 14:32:13 INFO DAGScheduler: ResultStage 165 (jdbc at <unknown>:0) finished in 0,248 s
24/10/10 14:32:13 INFO DAGScheduler: Job 123 is finished. Cancelling potential speculative or zombie tasks for this job
24/10/10 14:32:13 INFO TaskSchedulerImpl: Killing all running tasks in stage 165: Stage finished
24/10/1

24/10/10 14:42:10 INFO BlockManagerInfo: Removed broadcast_126_piece0 on 192.168.34.128:59436 in memory (size: 29.7 KiB, free: 434.3 MiB)
24/10/10 14:42:10 INFO BlockManagerInfo: Removed broadcast_128_piece0 on 192.168.34.128:59436 in memory (size: 32.5 KiB, free: 434.4 MiB)
24/10/10 14:42:10 INFO BlockManagerInfo: Removed broadcast_127_piece0 on 192.168.34.128:59436 in memory (size: 23.2 KiB, free: 434.4 MiB)
